# Jetson stats test

Graphic reference:
* http://urwid.org/examples/index.html
* https://npyscreen.readthedocs.io/
* https://github.com/chubin/cheat.sh


SUDO:
```console
RAM 1736/15827MB (lfb 3283x4MB) CPU [1%@1160,2%@1161,0%@1189,0%@1189,off,off,off,off] EMC_FREQ 0%@665 GR3D_FREQ 0%@114 APE 150 MTS fg 0% bg 0% AO@29C GPU@30C iwlwifi@34C Tboard@30C Tdiode@33C AUX@29C CPU@30C thermal@29.45C PMIC@100C GPU 466/466 CPU 466/466 SOC 932/932 CV 0/0 VDDRQ 0/0 SYS5V 1854/1854
```

NON SUDO:
```console
RAM 1733/15827MB (lfb 3283x4MB) CPU [1%@1190,0%@1190,0%@1190,0%@1190,off,off,off,off] EMC_FREQ 0% GR3D_FREQ 4% AO@25.5C GPU@26.5C iwlwifi@32C Tboard@27C Tdiode@29.75C AUX@25.5C CPU@26.5C thermal@26.1C PMIC@100C GPU 621/621 CPU 465/465 SOC 1397/1397 CV 0/0 VDDRQ 0/0 SYS5V 2053/2053
```

In [1]:
import subprocess, os
from termcolor import colored, cprint

In [2]:
test_stat_sudo = "RAM 1736/15827MB (lfb 3283x4MB) CPU [1%@1160,2%@1161,0%@1189,0%@1189,off,off,off,off] EMC_FREQ 0%@665 GR3D_FREQ 0%@114 APE 150 MTS fg 0% bg 0% AO@29C GPU@30C iwlwifi@34C Tboard@30C Tdiode@33C AUX@29C CPU@30C thermal@29.45C PMIC@100C GPU 466/466 CPU 466/466 SOC 932/932 CV 0/0 VDDRQ 0/0 SYS5V 1854/1854"
test_stat = "RAM 1733/15827MB (lfb 3283x4MB) CPU [1%@1190,0%@1190,0%@1190,0%@1190,off,off,off,off] EMC_FREQ 0% GR3D_FREQ 4% AO@25.5C GPU@26.5C iwlwifi@32C Tboard@27C Tdiode@29.75C AUX@25.5C CPU@26.5C thermal@26.1C PMIC@100C GPU 621/621 CPU 465/465 SOC 1397/1397 CV 0/0 VDDRQ 0/0 SYS5V 2053/205"
cprint(test_stat, 'red')

print("User: ", os.getuid())

RAM 1733/15827MB (lfb 3283x4MB) CPU [1%@1190,0%@1190,0%@1190,0%@1190,off,off,off,off] EMC_FREQ 0% GR3D_FREQ 4% AO@25.5C GPU@26.5C iwlwifi@32C Tboard@27C Tdiode@29.75C AUX@25.5C CPU@26.5C thermal@26.1C PMIC@100C GPU 621/621 CPU 465/465 SOC 1397/1397 CV 0/0 VDDRQ 0/0 SYS5V 2053/205
User:  1001


In [96]:
import re

def get_RAM_status(text):
    # RAM X/Y (lfb NxZ)
    # Largest Free Block (lfb) is a statistic about the memory allocator. 
    # It refers to the largest contiguous block of physical memory 
    # that can currently be allocated: at most 4 MB.
    # It can become smaller with memory fragmentation.
    # The physical allocations in virtual memory can be bigger.
    # X = Amount of RAM in use in MB.
    # Y = Total amount of RAM available for applications.
    # N = The number of free blocks of this size.
    # Z = is the size of the largest free block. 
    ram_string = re.search('RAM (.+?)B', text).group()
    lfb_string = re.search('\(lfb (.+?)\)', text).group()
    ram_stat = re.findall("\d+", ram_string)
    lfb_stat = re.findall("\d+", lfb_string)
    text = re.sub('RAM (.+?)\) ', '', text)
    return {
        'RAM' : {'used': ram_stat[0], 'total': ram_stat[1]},
        'lfb' : {'nblock': lfb_stat[0], 'size': lfb_stat[1]},
    }, text

def get_value_processor(text):
    if '@' in text:
        info = re.findall("\d+", text)
        return {'idle': int(info[0]), 'frequency': int(info[1])}
    else:
        info = re.findall("\d+", text)
        return {'idle': info[0]}
    return text

def get_CPU_status(text):
    # CPU [X%,Y%, , ]@Z
    # or
    # CPU [X%@Z, Y%@Z,...]
    # X and Y are rough approximations based on time spent
    # in the system idle process as reported by the Linux kernel in /proc/stat.
    # X = Load statistics for each of the CPU cores relative to the 
    #     current running frequency Z, or 'off' in case a core is currently powered down.
    # Y = Load statistics for each of the CPU cores relative to the 
    #     current running frequency Z, or 'off' in case a core is currently powered down.
    # Z = CPU frequency in megahertz. Goes up or down dynamically depending on the CPU workload.
    cpu_string = re.search('CPU (.+?)\]', text).group()
    cpu_string = cpu_string[cpu_string.find("[")+1:cpu_string.find("]")]
    text = re.sub('CPU (.+?)\] ', '', text)
    cpus = []
    for cpu in cpu_string.split(","):
        if 'off' in cpu:
            cpus.append(cpu)
        else:
            cpus.append(get_value_processor(cpu))
    
    return cpus, text

def get_status(text):
    jetsonstats = {}
    # Read RAM status
    ram_status, text = get_RAM_status(text)
    jetsonstats['RAM'] = ram_status
    # Read CPU status
    cpu_status, text = get_CPU_status(text)
    jetsonstats['CPU'] = cpu_status
    
    temperatures = {}
    volgates = {}
    idx = 0
    other_values = text.split(" ")
    while idx < len(other_values):
        data = other_values[idx]
        if 'EMC' in data:
            # EMC X%@Y
            # EMC is the external memory controller, 
            # through which all sysmem/carve-out/GART memory accesses go.
            # X = Percent of EMC memory bandwidth being used, relative to the current running frequency.
            # Y = EMC frequency in megahertz.
            jetsonstats['EMC'] = get_value_processor(other_values[idx+1])
            # extra increase counter
            idx += 1
        elif 'APE' in data:
            # APE Y
            # APE is the audio processing engine. 
            # The APE subsystem consists of ADSP (Cortex®-A9 CPU), mailboxes, AHUB, ADMA, etc.
            # Y = APE frequency in megahertz.
            jetsonstats['APE'] = other_values[idx+1]
            # extra increase counter
            idx += 1
        elif 'GR3D' in data:
            # GR3D X%@Y
            # GR3D is the GPU engine.
            # X = Percent of the GR3D that is being used, relative to the current running frequency.
            # Y = GR3D frequency in megahertz
            jetsonstats['GR3D'] = get_value_processor(other_values[idx+1])
            # extra increase counter
            idx += 1
        elif 'MTS' in data:
            # MTS fg X% bg Y%
            # X = Time spent in foreground tasks.
            # Y = Time spent in background tasks.
            fg = other_values[idx+2]
            bg = other_values[idx+4]
            jetsonstats['MTS'] = {'fg': fg, 'bg': bg}
            # extra increase counter
            idx += 4
        elif '@' in data:
            # [temp name] C
            # [temp name] is one of the names under the nodes
            # /sys/devices/virtual/thermal/thermal_zoneX/type.
            info = data.split("@")
            name = info[0]
            value = info[1]
            temperatures[name] = value
        else:
            # [VDD_name] X/Y
            # X = Current power consumption in milliwatts.
            # Y = Average power consumption in milliwatts.
            value = other_values[idx+1].split("/")
            volgates[data] = {'current': value[0], 'average': value[1]}
            # extra increase counter
            idx += 1
        # Update counter
        idx +=1
    
    jetsonstats['temperatures'] = temperatures
    jetsonstats['volgates'] = volgates
    jetsonstats['other'] = text
    
    return jetsonstats

In [97]:
#from jstatslib import get_status

def temp_show(gpu_stat):
    # Show result
    for key, value in gpu_stat.items():
        if isinstance(value, dict):
            print("[%s] "%key)
            for key_in, value_in in value.items():
                print("    [%s] "%key_in, value_in)
        elif isinstance(value, list):
            print("[%s] "%key)
            for data in value:
                print("    ", data)
        else:
            print("[%s]"%key, value)

# Original stat
cprint(test_stat_sudo, 'red')
# Test parser
gpu_stat = get_status(test_stat_sudo)
# Show result
temp_show(gpu_stat)

RAM 1736/15827MB (lfb 3283x4MB) CPU [1%@1160,2%@1161,0%@1189,0%@1189,off,off,off,off] EMC_FREQ 0%@665 GR3D_FREQ 0%@114 APE 150 MTS fg 0% bg 0% AO@29C GPU@30C iwlwifi@34C Tboard@30C Tdiode@33C AUX@29C CPU@30C thermal@29.45C PMIC@100C GPU 466/466 CPU 466/466 SOC 932/932 CV 0/0 VDDRQ 0/0 SYS5V 1854/1854
[RAM] 
    [RAM]  {'used': '1736', 'total': '15827'}
    [lfb]  {'nblock': '3283', 'size': '4'}
[CPU] 
     {'idle': 1, 'frequency': 1160}
     {'idle': 2, 'frequency': 1161}
     {'idle': 0, 'frequency': 1189}
     {'idle': 0, 'frequency': 1189}
     off
     off
     off
     off
[EMC] 
    [idle]  0
    [frequency]  665
[GR3D] 
    [idle]  0
    [frequency]  114
[APE] 150
[MTS] 
    [fg]  0%
    [bg]  0%
[temperatures] 
    [AO]  29C
    [GPU]  30C
    [iwlwifi]  34C
    [Tboard]  30C
    [Tdiode]  33C
    [AUX]  29C
    [CPU]  30C
    [thermal]  29.45C
    [PMIC]  100C
[volgates] 
    [GPU]  {'current': '466', 'average': '466'}
    [CPU]  {'current': '466', 'average': '466'}
    [SOC] 

In [ ]:
from IPython.display import display, clear_output

p = subprocess.Popen(['/home/nvidia/tegrastats'], stdout=subprocess.PIPE)
# Grab stdout line by line as it becomes available.  This will loop until 
# p terminates.
while p.poll() is None:
    clear_output(wait=True)
    # This blocks until it receives a newline.
    tegrastats_stream = p.stdout.readline().decode("utf-8")
    print(tegrastats_stream)
    gpu_stat = get_status(tegrastats_stream)
    
    temp_show(gpu_stat)
    
# When the subprocess terminates there might be unconsumed output 
# that still needs to be processed.
print(p.stdout.read())